In [1]:
import numpy as np
import torch as T
import pandas as pd

device = 'cpu'

### Обрабатываем данные и попутно удаляем оттуда элементы типа NaN  истороковые значения

In [2]:
data = pd.read_csv('Shanghai_HMT_2010.csv')
data = data.dropna() 
data = data.drop(['cbwd'], axis=1)

### Проведем нормализацию данных

In [3]:
data = (data - data.mean()) / data.std()

### Разбиение давления из data по среднему значение (mediana). Больше - присваивается индекс 1, меньше - индекс 0

In [4]:
pres_median = data['PRES'].median()
data['PRES'] = (data['PRES'] > pres_median).astype('int64')


## Проведем сравнение работы метода из sklearn и нашего собственного класса
### Начнем с sklearn

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

x = data.drop(['PRES'], axis=1)
y = data['PRES']

# установим random_state, чтобы результаты эксперимента были воспроизводимы
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

model = LogisticRegression()

# обучение
model.fit(x_train, y_train)

# предсказание
predictions_lr = model.predict(x_test)

# ошибка
error_sklearn = mean_squared_error(predictions_lr, y_test)
print("mean_squared_error =", error_sklearn)

mean_squared_error = 0.07789086796720385


### Теперь разберем наш класс

In [6]:
import torch as T
from MyClass import MyLogisticRegression

In [7]:
x = data.drop(['PRES'], axis=1)
y = data['PRES']

# установим random_state, чтобы результаты эксперимента были воспроизводимы
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
type(x_test)

pandas.core.frame.DataFrame

In [8]:
x_train_tenzor = T.tensor(x_train.to_numpy(), dtype=T.float32).to(device)
y_train_tenzor = T.tensor(y_train.to_numpy(), dtype=T.long).to(device)


x_test_tenzor = T.tensor(x_test.to_numpy(), dtype=T.float32).to(device)
y_test_tenzor = T.tensor(y_test.to_numpy(), dtype=T.long).to(device)

num_of_features = x_train_tenzor.size()[1]

In [9]:
mymodel = MyLogisticRegression(num_of_features)
times = 12
w, b = mymodel.fit(times, x_train_tenzor, y_train_tenzor)

epoch =    0    loss = 0.2482
epoch =    4    loss = 0.1035
epoch =    8    loss = 0.0885


In [10]:
x_test_tenzor = T.tensor(x_test.to_numpy(), dtype=T.float32).to(device)
predicts = mymodel.predictions(x_test_tenzor)

In [11]:
error = mean_squared_error(predicts, y_test)
print("mean_squared_error =", error)

mean_squared_error = 0.07879420790545504
